Cette feuille a pour but de travailler a la construction d'un AST

Ne pas oublier :
lorsque l'AST est construit :
    - 1 seul token equals
    - 1 partie droit existante
    - rajouter les prochains trucs a verifier a posteriori

In [416]:
class AST(object):
    pass

class Node_letter(object):
    
    def __init__(self, token, name):
        self.token = token
        self.name = name
        self.state = 0
        self.neg = 0
        self.childs_pos = []
        self.childs_neg = []
        
    def __str__(self):
        return "Node_letter({})".format(self.token)
        
class Node_condition(object):
    
    def __init__(self, left, cond, right):
        self.left = left
        self.token = self.cond = cond
        self.right = right
        self.neg = 0
        
    def __str__(self):
        return "Node_condition({}".format(self.token)
class Token(object):
    def __init__(self, token_type, value):
        self.type = token_type
        self.value = value

    def __str__(self):
        """String representation of the class instance.
        Examples:
            Token(NEG, "!")
            Token(LETTER, "A")
            Token(AND, '+')
            Token(OR, '|')
            Token(XOR, '^')
        """
        return 'Token({type}, {value})'.format(
            type=self.type,
            value=repr(self.value)
        )

    def __repr__(self):
        return self.__str__()            

In [254]:
class Lexer(object):
    def __init__(self, line):
        self.rule = line
        
        self.pos = 0
        self.current_char = self.rule[self.pos]
    
    def error(self, EOL=False):
        if EOL == False:
            raise Exception('Invalid character \'{}\' at index {}'.format(self.rule[self.pos], self.pos + 1))
        else:
            raise Exception('Invalid character \'{}\' at index {}'.format("EOL", self.pos + 1))
    def get_next_token(self):
        dic_op = {
            "+": "AND",
            "|": "OR",
            "^": "XOR",
            "!": "NOT",
            "(": "OPEN_PAR",
            ")": "CLOSE_PAR"
        }
        dic_equal = {
            "=": "EQUAL",
            ">": "IMPLIES",
            "<": "ONLY_IF"
        }
        i = 0
        len_rule = len(self.rule)
        while i <  len_rule:
            self.pos = i
            if self.rule[i] == " ":
                i += 1
                continue
            if ord(self.rule[i]) in range(ord("A"), ord("Z") + 1):
                yield Token("LETTER", self.rule[i])
            elif self.rule[i] in dic_op:
                yield Token(dic_op[self.rule[i]], self.rule[i])
            elif self.rule[i] in dic_equal:
                tmp = self.get_equals_token(i, len_rule)
                i += tmp[0]
                yield tmp[1]
            else:
                self.error()
            i += 1
        yield Token("EOL", None)
    
    def get_equals_token(self, i, len_rule):
        if self.rule[i] == "<" and i < len_rule - 2:
            if self.rule[i + 1] == "=":
                if self.rule[i + 2] == ">":
                    return ([2, Token("ONLY_IF", "<=>")])
                else:
                    self.pos += 2
                    self.error()
            else:
                self.pos += 1
                self.error()
        elif self.rule[i] == "=" and i < len_rule - 1:
            if self.rule[i + 1] == ">":
                return ([1, Token("IMPLIES", "=>")])
            else:
                self.pos += 1
                self.error()
        if i == len_rule - 1:
            self.pos += 1
            self.error(EOL=True)
        elif i == len_rule - 2:
            self.pos += 2
            self.error(EOL=True)
        self.error()

le principe du Parser repose sur de la recursivitée, en impliquant des priorité sur les tokens rencontrés.

Dans notre problème, on peut distinguer 3 niveaux de priorités, du moins prioritaire au plus prioritaire

* 1: le token equal. (=> ou <=>)
* 2: operateurs : ^, |, +
* 3: lettres, parentheses

* A noter que la negation n'est pas encore bien integrée a l'arbre


la recursivitée est programmée comme ceci :
le niveau 1 apelle le niveau 2 qui apelle le niveau 3

In [3]:
class Parser(object):
    def __init__(self, lexer):
        self.lexer = lexer
        self.gen = self.lexer.get_next_token()
        self.current_token = next(self.gen)
        
    def error(self, s):
        raise Exception("Invalid syntax : {}".format(s))
    
    def get_next_token(self, token_type):
        if self.current_token.type == token_type:
            self.current_token = next(self.gen)
        else:
            self.error("could not find match of parenthesis")
    def parse(self):
        """
        Cette fonction a pour but d'enclencher le debut du parsing recursif.
        la variable node va etre le neud representant la racine de l'AST.
        """
        node = self.deep_one()
        if self.current_token.type != "EOL":
            self.error(self.lexer.error())
        if (node.token.type != "IMPLIES") and (node.token.type != "ONLY_IF"):
            self.error("Missing valid equals sign")
        return node
    
    
    def deep_one(self):
        """
        Cette pronfondeur de prioritée est assignée aux token suivants :
            Token(IMPLIES, '=>')
            Token(ONLY_IF, '<=>')            
            Token(NEG, '!')
        """
        node = self.deep_two()
        
        
        while self.current_token.type in ("IMPLIES", "ONLY_IF"):
            token = self.current_token
            if token.type == "IMPLIES":
                self.get_next_token("IMPLIES")
            if token.type == "ONLY_IF":
                self.get_next_token("ONLY_IF")
                
            node = Node_condition(left=node, cond=token, right=self.deep_two())
            
        return node
    
    def deep_two(self):
        """
        Cette profondeur de prioritée est assignée aux token suivants :
            Token(AND, '+')
            Token(OR, '|')
            Token(XOR, '^')
        """
        node = self.deep_three()
        
        while self.current_token.type in ("AND", "OR", "XOR"):
            token = self.current_token
            if token.type == "AND":
                self.get_next_token("AND")
            if token.type == "OR":
                self.get_next_token("OR")
            if token.type == "XOR":
                self.get_next_token("XOR")
            
            node = Node_condition(left=node, cond=token, right=self.deep_three())
        return node
    
    
    def deep_three(self):
        """
        Cette profondeur de prioritée est assignée au token equals :
            Token(LETTER, 'A->Z')
            Token(OPEN_PAR, '(' )
            Token(CLOSE_PAR, ')' )
            Token(NOT, '!')
        """
        token = self.current_token
        if token.type == "LETTER":
            self.get_next_token("LETTER")
            return Node_letter(token, token.value)
        elif token.type == "OPEN_PAR":
            self.get_next_token("OPEN_PAR")
            node = self.deep_one()
            self.get_next_token("CLOSE_PAR")
            return node
        elif token.type == "NOT":
            self.get_next_token("NOT")
            token = self.current_token
            if token.type == "LETTER":
                self.get_next_token("LETTER")
                ret = Node_letter(token, token.value)
                ret.neg = 1
                return ret
            elif token.type == "OPEN_PAR":
                self.get_next_token("OPEN_PAR")
                node = self.deep_one()
                node.neg = 1
                self.get_next_token("CLOSE_PAR")
                return node
        else:
            self.lexer.error()

In [421]:
class Rule_interpreter(object):
    
    def __init__(self):
        pass
#         self.root = root
#         self.set_facts(root, true_facts)
        
    def interpret(self, node):
        if node:
            if type(node).__name__ == "Node_condition":
                return abs(self.apply_logical(node, node.left, node.right) - node.neg)
            if type(node).__name__ == "Node_letter":
                for child in (node.childs_pos + node.childs_neg):
                    if self.interpret(child) == 1:
                        node.state = 1
#                 for child in node.childs_neg: # ZIP HERE
#                     if self.interpret(child) == 1:
#                         node.state
                return abs(node.state - node.neg)
    
    def apply_logical(self, node, left, right):
        if node.token.type == "AND":
            return self.interpret(left) & self.interpret(right)
        if node.token.type == "OR":
            return self.interpret(left) | self.interpret(right)
        if node.token.type == "XOR":
            return self.interpret(left) ^ self.interpret(right)
    
    def set_facts(self, node, true_facts):
        if node:
            if type(node).__name__ == "Node_letter":
                if node.token.value in true_facts:
                    node.state = 1
            if type(node).__name__ == "Node_condition":
                self.set_facts(node.left, true_facts)
                self.set_facts(node.right, true_facts)
                

In [71]:
rule1 = "A ^ B => D"
rule2 = "A + B => D"
true_facts = "AB"
try:
    lexer = Lexer(rule1)
    parser = Parser(lexer)
    root = parser.parse()
    interpretor = Rule_interpreter(root.left, true_facts)
    print(interpretor.interpret(root.left))
except Exception as e:
    print(e)

__init__() takes 1 positional argument but 3 were given


In [452]:
class Graph(object):
    
    def __init__(self, true_facts):
        self.implies_list = []
        self.implies_name_list = []
        self.true_facts = true_facts
        self.interpretor = Rule_interpreter()
        
    def add_new_AST(self, root):
        self.update_implies_list(root.right)
        self.update_graph(root.left, root.right)
        
    
    def update_implies_list(self, node):
        if node:
            if type(node).__name__ == "Node_letter":
                if node.name not in self.implies_name_list:
                    self.implies_list.append(node)
                    self.implies_name_list.append(node.name)
            if type(node).__name__ == "Node_condition":
                self.update_implies_list(node.left)
                self.update_implies_list(node.right)
    
    def update_graph(self, left, node):
        if node:
            if type(node).__name__ == "Node_letter":
                if node.neg == 1:
                    self.find_letter_in_implies(node.name).childs_neg.append(left)
                else:
                    self.find_letter_in_implies(node.name).childs_pos.append(left)
            if type(node).__name__ == "Node_condition":
                self.update_graph(left, node.left)
                self.update_graph(left, node.right)
    
        
    def find_letter_in_implies(self, name):
        for node in self.implies_list:
            if node.name == name:
                return node
        raise Exception("Error node not found")
    

    def set_facts(self, node):
        if node:
            if type(node).__name__ == "Node_letter":
                for child in (node.childs_pos + node.childs_neg):
                    self.set_facts(child)
                if node.token.value in self.true_facts:
                    node.state = 1
            if type(node).__name__ == "Node_condition":
                self.set_facts(node.left)
                self.set_facts(node.right)
                
    
    def query(self, letter):
        if letter in self.full_history.keys():
            print("{} is {}".format(letter, self.full_history[letter]))
            return self.full_history[letter]
        elif letter in self.true_facts:
            print("{} is True".format(letter))
            return True
        else:
            print("{} is False".format(letter))
            return False
    
    def check_contradiction(self):
        self.full_history = {}
        for node in self.implies_list:
            self.full_history[node.name] = self.get_final_state(node)
            if self.full_history[node.name] == None:
                if node.name in self.true_facts:
                    self.full_history[node.name] = True
                else:
                    self.full_history[node.name] = False
                    
    def get_final_state(self, node):
        history = []
        for child in node.childs_pos:
            res = self.interpretor.interpret(child)
            if res == 1:
                history.append(True)
                
        for child in node.childs_neg:
            res = self.interpretor.interpret(child)
            if res == 1:
                history.append(False)
            
        if len(set(history)) == 0:
            return
        elif len(set(history)) == 1:
            return history[0]
        else:
            raise Exception("Error contradiction found with letter {}".format(node.name))
    

In [453]:
rule1 = "A + B => D"
# rule1bis = "B + C => !D"
# rule1ter = "(B + C) | A => D"
rule2 = "E + C => B + !A"
rule3 = "G => H"

# rule1 = "A + B => !C"
# rule2 = "D + E => F"
# rule3 = "G + H => I"
# rule2 = "C => A"


true_facts = "AEC"

try:
    graph = Graph(true_facts)
    
    root1 = Parser(Lexer(rule1)).parse()
    graph.add_new_AST(root1)
    
    root2 = Parser(Lexer(rule2)).parse()
    graph.add_new_AST(root2)
    
    root3 = Parser(Lexer(rule3)).parse()
    graph.add_new_AST(root3)
    
    for node in graph.implies_list:
        graph.set_facts(node)
    
    graph.check_contradiction()
    
    graph.query("A")
    graph.query("B")
    graph.query("F")
    graph.query("D")
except Exception as e:
    print(e)

A is False
B is True
F is False
D is False


In [402]:
for c in graph.implies_list:
    print(c, c.name, c.neg)
# print(graph.implies_list[0].childs[0].left)
# print(graph.check_contrast())
# print()
# for c in graph.implies_list[0].childs:
#     print(c.left, c.left.childs[0].opp)

Node_letter(Token(LETTER, 'D')) D 0
Node_letter(Token(LETTER, 'B')) B 0
Node_letter(Token(LETTER, 'A')) A 1
Node_letter(Token(LETTER, 'H')) H 0


In [348]:
prefix_run(graph.implies_list[0])

Node_letter(Token(LETTER, 'D')) 0
Node_condition(Token(AND, '+') 0
Node_letter(Token(LETTER, 'A')) 0
Node_condition(Token(AND, '+') 1
Node_letter(Token(LETTER, 'E')) 0
Node_letter(Token(LETTER, 'C')) 0
Node_letter(Token(LETTER, 'B')) 0
Node_condition(Token(AND, '+') 1
Node_letter(Token(LETTER, 'E')) 0
Node_letter(Token(LETTER, 'C')) 0
Node_condition(Token(AND, '+') 0
Node_letter(Token(LETTER, 'A')) 0
Node_condition(Token(AND, '+') 1
Node_letter(Token(LETTER, 'E')) 0
Node_letter(Token(LETTER, 'C')) 0
Node_letter(Token(LETTER, 'B')) 0
Node_condition(Token(AND, '+') 1
Node_letter(Token(LETTER, 'E')) 0
Node_letter(Token(LETTER, 'C')) 0


In [347]:
def prefix_run(node):
    if node:
        print(node, node.opp)
        if type(node).__name__ == "Node_letter":
            for c in node.childs:
                prefix_run(c)
        if type(node).__name__ == "Node_condition":
            prefix_run(node.left)
            prefix_run(node.right)

In [443]:
a = {}
a["A"] = 12
if "B" in a.keys():
    print("ok")

In [440]:

# rule1c = "A ^ B => !D"
# rule2 = "(A+B) | (A ^ B) => D"

rule3 = "A + B => D"
rule4 = "A ^ B => D"
rules5 = "A => !B"
ruleA = "!A => B"
ruleB = "A => B"
try:
    root1 = Parser(Lexer(rule1)).parse()
    root2 = Parser(Lexer(rule2)).parse()
except Exception as e:
    print(e)